In [1]:
from apache_atlas.client.ApacheAtlas import ApacheAtlasClient

atlas_section = ApacheAtlasClient(
    "http://10.100.100.61:21000",
    "admin",
    "admin"
)

## Criação das Tabelas

In [2]:
groups_sia ={
    #'AB': 'APAC de Cirurgia Bariátrica',
    'ABO': 'APAC de Acompanhamento Pós Cirurgia Bariátrica',
    'ACF': 'APAC de Confecção de Fístula',
    'AD': 'APAC de Laudos Diversos',
    'AM': 'APAC de Medicamentos',
    'AMP': 'APAC de Acompanhamento Multiprofissional',
    'AN': 'APAC de Nefrologia',
    'AQ': 'APAC de Quimioterapia',
    'AR': 'APAC de Radioterapia',
    'ATD': 'APAC de Tratamento Dialítico',
    'BI': 'Boletim de Produção Ambulatorial individualizado',
    'PA': 'Produção Ambulatorial',
    'PS': 'RAAS Psicossocial',
    'SAD': 'RAAS de Atenção Domiciliar'
}

for acronymus, name in groups_sia.items():
    atlas_section.entity.create_entity_dt_table({
       'name': f"{name} ({acronymus})",
       'acronymus': acronymus,
       'description': f'Tabela de {name} da SIA' 
    }, 'SIA')

http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_database&attrName=acronymus&attrValuePrefix=SIA&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_database&attrName=acronymus&attrValuePrefix=SIA&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_database&attrName=acronymus&attrValuePrefix=SIA&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_database&attrName=acronymus&attrValuePrefix=SIA&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_database&attrName=acronymus&attrValuePrefix=SIA&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_database&attrName=acr

In [3]:
atlas_section.entity.create_entity_columns("dicionarios/columns-AQ.csv", "AQ")

http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table&attrName=acronymus&attrValuePrefix=AQ&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/bulk


[{'typeName': 'dt_table_column',
  'attributes': {'owner': '',
   'qualifiedName': 'dt_table_column.DataSUS.AQ@AP_MVM',
   'name': 'AP_MVM',
   'description': 'Data de Processamento / Movimento (AAAAMM)'},
  'guid': 'cc665736-9af4-4ba6-9369-0f1e7c5940aa',
  'status': 'ACTIVE',
  'displayText': 'AP_MVM',
  'classificationNames': [],
  'classifications': [],
  'meaningNames': [],
  'meanings': [],
  'isIncomplete': False,
  'labels': []},
 {'typeName': 'dt_table_column',
  'attributes': {'owner': '',
   'qualifiedName': 'dt_table_column.DataSUS.AQ@AP_CONDIC',
   'name': 'AP_CONDIC',
   'description': 'Sigla do Tipo de Gestão no qual o Estado ou Município está habilitado'},
  'guid': 'd59d8373-7dc3-40c2-a407-e1dd394a2b52',
  'status': 'ACTIVE',
  'displayText': 'AP_CONDIC',
  'classificationNames': [],
  'classifications': [],
  'meaningNames': [],
  'meanings': [],
  'isIncomplete': False,
  'labels': []},
 {'typeName': 'dt_table_column',
  'attributes': {'owner': '',
   'qualifiedName':

In [ ]:
data = {
"AQ0803":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0804":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0805":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0806":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0807":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0808":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0809":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0810":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0811":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0812":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0901":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0902":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0903":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0904":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0905":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0906":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0907":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0908":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0909":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0910":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0911":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ0912":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1001":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1002":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1003":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1004":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1005":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1006":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1007":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1008":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1009":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1010":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1011":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1012":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1101":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1102":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1103":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1104":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1105":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1106":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1107":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1108":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1109":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1110":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1111":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1112":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1201":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1202":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1203":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1204":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1205":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1206":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1207":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1208":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1209":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1210":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1211":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1212":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1301":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1302":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1303":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1304":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1305":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1306":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1307":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1308":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1309":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1310":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1311":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1312":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1401":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1402":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1403":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1404":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1405":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1406":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1407":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1408":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1409":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1410":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1411":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1412":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1501":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1502":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1503":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1504":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1505":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1506":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1507":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1508":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1509":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1510":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1511":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1512":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1601":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1602":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1603":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1604":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1605":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1606":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2'],
"AQ1607":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1608":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1609":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1610":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1611":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1612":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1701":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1702":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1703":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1704":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1705":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1706":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1707":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1708":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1709":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1710":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1711":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1712":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1801":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1802":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1803":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1804":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1805":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1806":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1807":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1808":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1809":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1810":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1811":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1812":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1901":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1902":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1903":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1904":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1905":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1906":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1907":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1908":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1909":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1910":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1911":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ1912":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2001":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2002":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2003":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2004":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2005":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2006":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2007":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2008":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2009":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2010":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2011":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2012":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AP_NATJUR'],
"AQ2101":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2102":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2103":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2104":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2105":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2106":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2107":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2108":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2109":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2110":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2111":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2112":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2201":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2202":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2203":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2204":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2205":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2206":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2207":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2208":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2209":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2210":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2211":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2212":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2301":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2302":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2303":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2304":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2305":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2306":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2307":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2308":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2309":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2310":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2311":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2312":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2401":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2402":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2403":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2404":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2405":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2406":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR'],
"AQ2407":  ['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR']
}

# Só aceitar no formato TTYYMM onde TT é a sigla da tabela, YY o ano e MM o mes
atlas_section.lineage.create_lineage_table(data, "BI")